In [77]:
import pandas as pd
import datetime as dt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import math
import warnings
warnings.filterwarnings('ignore')

In [78]:
pd.__version__

'1.4.2'

In [79]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')

In [80]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [81]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.0,4.39,1.0,N,68,236,1,20.5,1.0,0.5,1.28,0.00,1.0,26.78,2.5,0.00
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.0,7.71,1.0,N,48,243,1,31.0,1.0,0.5,9.00,0.00,1.0,45.00,2.5,0.00
2,2,2024-02-01 00:07:50,2024-02-01 00:43:12,2.0,28.69,2.0,N,132,261,2,70.0,0.0,0.5,0.00,6.94,1.0,82.69,2.5,1.75
3,1,2024-02-01 00:01:49,2024-02-01 00:10:47,1.0,1.10,1.0,N,161,163,1,9.3,3.5,0.5,2.85,0.00,1.0,17.15,2.5,0.00
4,1,2024-02-01 00:37:35,2024-02-01 00:51:15,1.0,2.60,1.0,N,246,79,2,15.6,3.5,0.5,0.00,0.00,1.0,20.60,2.5,0.00


In [82]:
print("#columns in the extract:", len(df_jan.columns))

#columns in the extract: 19


In [83]:
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds().div(60)
print("Standard Deviation of duration:", df_jan['duration'].std() )

Standard Deviation of duration: 34.851053592192876


In [84]:
df_jan['outlier'] = df_jan['duration'].apply(lambda x: 1 if x<1 or x>60 else 0 ) 

df_jan['outlier'].value_counts()

0    2898906
1      65718
Name: outlier, dtype: int64

In [85]:
df_jan_cleaned=df_jan[df_jan['outlier']==0]
df_jan_cleaned['outlier'].value_counts()

0    2898906
Name: outlier, dtype: int64

In [86]:
print('remaining records in percentage: ', df_jan_cleaned.shape[0]/df_jan.shape[0])

remaining records in percentage:  0.9778326020432945


In [87]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

In [88]:
df_jan_cleaned[categorical].dtypes

PULocationID    int32
DOLocationID    int32
dtype: object

In [106]:
df_jan_cleaned[categorical] = df_jan_cleaned[categorical].astype(str)

In [90]:
df_jan_cleaned[categorical].dtypes

PULocationID    object
DOLocationID    object
dtype: object

In [91]:
train_dicts = df_jan_cleaned[categorical + numerical].to_dict(orient = 'records')

In [92]:
dv = DictVectorizer(sparse = True)
X_train = dv.fit_transform(train_dicts)

In [93]:
X_train

<2898906x519 sparse matrix of type '<class 'numpy.float64'>'
	with 8696718 stored elements in Compressed Sparse Row format>

In [94]:
y_train = df_jan_cleaned[target].values
y_train

array([19.8       ,  6.6       , 17.91666667, ..., 19.        ,
       17.85      , 15.08333333])

In [95]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [96]:
y_predict = lr.predict(X_train)

In [97]:
rmse = math.sqrt(metrics.mean_squared_error(y_train, y_predict))
print("Mean Squared Error for train data:", rmse)

Mean Squared Error for train data: 7.946006590881357


In [107]:
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds().div(60)
df_feb['outlier'] = df_feb['duration'].apply(lambda x: 1 if x<1 or x>60 else 0 )
df_feb_cleaned=df_feb[df_feb['outlier']==0]
df_feb_cleaned[categorical] = df_feb_cleaned[categorical].astype(str)

In [104]:
y_val = df_feb_cleaned[target].values
val_dicts = df_feb_cleaned[categorical + numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)
y_val_predict = lr.predict(X_val)
rmse = math.sqrt(metrics.mean_squared_error(y_val, y_val_predict))
print("Mean Squared Error for Validation data:", rmse)

Mean Squared Error for Validation data: 8.123198435285437
